In [3]:
from pyspark.sql import SparkSession
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'secrets/serviceKey.json'


# JAR paths for BigQuery and GCS connectors
bigquery_connector_jar = "spark-bigquery-connector.jar"
gcs_connector_jar = "gcs-connector.jar"


# Create SparkSession with both connectors
spark = SparkSession.builder \
    .appName("PySpark with BigQuery and GCS") \
    .config("spark.jars", f"{bigquery_connector_jar},{gcs_connector_jar}") \
    .config("spark.sql.catalog.spark_bigquery", "com.google.cloud.spark.bigquery.BigQueryCatalog") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "secrets/serviceKey.json") \
    .config("spark.bigquery.projectId", "idmpproject-441123") \
    .getOrCreate()

spark

24/11/20 16:35:34 WARN Utils: Your hostname, Rishis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.110.25.231 instead (on interface en0)
24/11/20 16:35:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/11/20 16:35:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Exception in thread "main" java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.hadoop.fs.FileSystem$Cache$Key.<init>(FileSystem.java:3746)
	at org.apache.hadoop.fs.FileSystem$Cache$Key.<init>(FileSystem.java:3736)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3520)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.spark.util.Depen

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [5]:
# Specify the BigQuery table
project_id = "idmpproject-441123"
dataset_id = "uberFareEstimation"
table_name = "uber_data"

bigquery_table = f"{project_id}.{dataset_id}.{table_name}"

# Read data from BigQuery into a Spark DataFrame
uber_df = spark.read \
    .format("bigquery") \
    .option("table", bigquery_table) \
    .load()

# Show the first few rows of the DataFrame
uber_df.show()

NameError: name 'spark' is not defined

In [6]:
table_name = "lyft_data"

bigquery_table = f"{project_id}.{dataset_id}.{table_name}"

# Read data from BigQuery into a Spark DataFrame
lyft_df = spark.read \
    .format("bigquery") \
    .option("table", bigquery_table) \
    .load()

# Show the first few rows of the DataFrame
lyft_df.show()

NameError: name 'spark' is not defined

In [7]:
table_name = "weather_data"

bigquery_table = f"{project_id}.{dataset_id}.{table_name}"

# Read data from BigQuery into a Spark DataFrame
weather_df = spark.read \
    .format("bigquery") \
    .option("table", bigquery_table) \
    .load()

# Show the first few rows of the DataFrame
weather_df.show()

NameError: name 'spark' is not defined

In [1]:
from pyspark.sql.functions import col, lit
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the 90th percentile value for price
percentile_90 = uber_df.approxQuantile("price", [0.99], 0.001)[0]

# Filter the DataFrame to include only the first 90% of prices
filtered_data = uber_df.filter(col("price") <= lit(percentile_90))

# Convert the filtered data to Pandas
filtered_price_data = filtered_data.select("price").toPandas()

# Plot Histogram for the first 90% of prices
plt.figure(figsize=(10, 6))
sns.histplot(filtered_price_data['price'], bins=30, kde=True, color='blue')
plt.title('Distribution of Fare Prices (First 90% of Data)')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

# Plot Boxplot for the first 90% of prices
plt.figure(figsize=(8, 4))
sns.boxplot(x=filtered_price_data['price'], color='cyan')
plt.title('Boxplot of Fare Prices (First 90% of Data)')
plt.xlabel('Price')
plt.show()

NameError: name 'uber_df' is not defined

In [ ]:
# Visualization 2 Distance vs Price (scatter plot)

distance_price_df = filtered_data.select('distance', 'price').toPandas()

plt.figure(figsize=(10, 6))
sns.scatterplot(x=distance_price_df['distance'], y=distance_price_df['price'], hue='price')
plt.title('Relationship between Distance and Price')
plt.xlabel('Distance (Miles)')
plt.ylabel('Price (USD')
plt.show()
